In [ ]:
import os
import logging
from typing import List, Dict, Union
from datetime import datetime
import time
import json
from huggingface_hub import InferenceClient
import re

# Basic setup
logging.basicConfig(level=logging.INFO)


In [ ]:
# Configuration for AIP models
API_CONFIGS = {
    "mixtral": {
        "url": "https://mixtral-8x22b.k8s.aip.mitre.org",
        "max_tokens": 60000,
        "batch_size": 3,
        "delay_between_chunks": 2,
        "delay_between_batches": 10
    },
    "llama": {
        "url": "https://llama3-70b.k8s.aip.mitre.org",
        "max_tokens": 4192,
        "batch_size": 3,
        "delay_between_chunks": 3,
        "delay_between_batches": 15
    },
    "codestral": {
        "url": "https://codestral-22b.k8s.aip.mitre.org",
        "max_tokens": 4192,
        "batch_size": 3,
        "delay_between_chunks": 2,
        "delay_between_batches": 10
    },
    "cohere": {
        "url": "https://command-rplus.k8s.aip.mitre.org",
        "max_tokens": 4192,
        "batch_size": 3,
        "delay_between_chunks": 2,
        "delay_between_batches": 10
    }
}


In [ ]:
def clean_markdown(text: str) -> str:
    """Clean markdown content"""
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    text = re.sub(r'\.{2,}', '.', text)
    text = re.sub(r'\\(.)', r'\1', text)
    text = re.sub(r'\|', ' ', text)
    text = re.sub(r'[-\s]*\n[-\s]*', '\n', text)
    return text.strip()

def split_markdown(content: str, max_size: int = 2000) -> List[str]:
    """Split markdown into manageable chunks"""
    chunks = []
    lines = content.split('\n')
    current_chunk = []
    current_size = 0
    
    for line in lines:
        line_size = len(line)
        if current_size + line_size > max_size:
            if current_chunk:
                chunks.append('\n'.join(current_chunk))
            current_chunk = [line]
            current_size = line_size
        else:
            current_chunk.append(line)
            current_size += line_size
            
    if current_chunk:
        chunks.append('\n'.join(current_chunk))
    return chunks

def create_client(model_type: str) -> InferenceClient:
    """Create a client for the specified model"""
    return InferenceClient(model=API_CONFIGS[model_type]["url"])

def process_markdown_chunk(client: InferenceClient, chunk: str, model_type: str) -> str:
    """Process a single markdown chunk with comprehensive requirements analysis"""
    prompt = f"""Analyze this FHIR Implementation Guide markdown content. Focus on requirements that would be included in a test kit to verify the behaviors of the system under test for this IG. A requirement defines how a system or user may or must behave.

Content to analyze:
{chunk}

The following information is needed about each requirement for the test kit:
1. Requirement statement text. Clarifications and context may be included within brackets to clearly distinguish source quotes from added detail
2. Sub-requirements: A reference to a list of requirements from this or another set that this requirement points to.
3. Conformance: The conformance level (SHALL, SHOULD, MAY, SHOULD NOT, SHALL NOT) indicating whether systems implementing the requirement can choose to meet it or not. The conformance level helps drive decisions about which requirements to test and which to prioritize creating tests for during test kit planning and development.
4. Actor(s): The actor, or actors, that this requirement applies to. Inferno test suites verify the behavior of specific systems that play the role of actors defined in the requirements. Only requirements that apply to the actor(s) played by the system under test need to be verified. Thus, pulling out the actor discretely enables the requirements coverage analysis to determine whether a requirement is in scope for a suite based on the actor(s) it verifies. It also assists with planning by indicating which requirements Inferno will need to verify and which it will need to simulate so it can interact with a given actor to gather the information needed to perform verification of other requirements.
5. Conditionality: Optional indicator of whether the requirement is conditional on another decision made by the system under test, such as whether to implement another optional requirement.

Extract precise requirements following these guidelines, focusing on new information not covered in previous chunks."""

    messages = [{"role": "user", "content": prompt}]
    response = client.chat_completion(messages, max_tokens=API_CONFIGS[model_type]["max_tokens"])
    return response

def process_markdown_content(markdown_dir: str, model_type: str) -> Dict:
    """Process all markdown content using specified model"""
    client = create_client(model_type)
    results = {
        "metadata": {
            "model": model_type,
            "timestamp": datetime.now().isoformat(),
            "processed_files": []
        },
        "summaries": []
    }
    
    for filename in os.listdir(markdown_dir):
        if filename.endswith('.md'):
            file_path = os.path.join(markdown_dir, filename)
            with open(file_path, 'r') as f:
                content = clean_markdown(f.read())
                
            chunks = split_markdown(content)
            chunk_summaries = []
            
            for chunk in chunks:
                summary = process_markdown_chunk(client, chunk, model_type)
                chunk_summaries.append(summary)
                time.sleep(API_CONFIGS[model_type]["delay_between_chunks"])
                
            results["metadata"]["processed_files"].append(filename)
            results["summaries"].extend(chunk_summaries)
            time.sleep(API_CONFIGS[model_type]["delay_between_batches"])
            
    return results


In [ ]:
def create_meta_summary(results: Dict, model_type: str) -> str:
    """Create a meta-summary from all processed content with enhanced focus on testing requirements"""
    client = create_client(model_type)
    
    meta_prompt = f"""Synthesize these content summaries into a comprehensive list outlining testing requirements, including information about conformance, actors, and conditionality:

{json.dumps(results['summaries'], indent=2)}

Create a comprehensive analysis that:
1. Eliminates redundant information
2. Maintains technical accuracy
3. Includes specific technical information about:
   3.1 Requirement statement text. Clarifications and context may be included within brackets to clearly distinguish source quotes from added detail
   3.2 Sub-requirements: A reference to a list of requirements from this or another set that this requirement points to.
   3.3 Conformance: The conformance level (SHALL, SHOULD, MAY, SHOULD NOT, SHALL NOT) indicating whether systems implementing the requirement can choose to meet it or not. The conformance level helps drive decisions about which requirements to test and which to prioritize creating tests for during test kit planning and development.
   3.4 Actor(s): The actor, or actors, that this requirement applies to. Inferno test suites verify the behavior of specific systems that play the role of actors defined in the requirements. Only requirements that apply to the actor(s) played by the system under test need to be verified.
   3.5 Conditionality: Optional indicator of whether the requirement is conditional on another decision made by the system under test, such as whether to implement another optional requirement.

Focus on extracting precise, testable requirements that would be needed for test kit development."""

    messages = [{"role": "user", "content": meta_prompt}]
    return client.chat_completion(messages, max_tokens=API_CONFIGS[model_type]["max_tokens"])


In [ ]:
# Setup directories
markdown_dir = "full-ig/markdown"
output_dir = "analysis_results"
os.makedirs(output_dir, exist_ok=True)

# Process with each model
models = ["mixtral", "llama", "codestral", "cohere"]

for model in models:
    try:
        logging.info(f"Processing with {model}...")
        
        # Process all markdown content
        results = process_markdown_content(markdown_dir, model)
        
        # Generate meta-summary
        meta_summary = create_meta_summary(results, model)
        results["meta_summary"] = meta_summary
        
        # Save results
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = os.path.join(output_dir, f"{model}_analysis_{timestamp}.json")
        
        with open(output_file, 'w') as f:
            json.dump(results, f, indent=2)
            
        logging.info(f"Results saved to: {output_file}")
        
    except Exception as e:
        logging.error(f"Error processing {model}: {str(e)}")
        continue